<a href="https://colab.research.google.com/github/darshan0309/Active-_Learning/blob/main/A3Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy

(_, _), (x_test, _) = cifar10.load_data()
x_test = x_test / 255.0

model = load_model('cifar10_model.h5')


def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))


def least_confidence(predictions):
    return 1 - np.max(predictions, axis=1)

def prediction_entropy(predictions):
    return entropy(predictions.T)

def margin_sampling(predictions):
    sorted_probs = np.sort(predictions, axis=1)
    return sorted_probs[:, -1] - sorted_probs[:, -2]

def contrastive_active_learning(predictions, k=5):
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(predictions)
    distances, _ = knn.kneighbors(predictions)
    return np.mean(distances[:, 1:], axis=1)


predictions = model.predict(x_test)


lc_uncertainty = least_confidence(predictions)
pe_uncertainty = prediction_entropy(predictions)
ms_uncertainty = margin_sampling(predictions)
cal_uncertainty = contrastive_active_learning(predictions)


print("Least Confidence Uncertainty:", lc_uncertainty)
print("Prediction Entropy Uncertainty:", pe_uncertainty)
print("Margin Sampling Uncertainty:", ms_uncertainty)
print("Contrastive Active Learning Uncertainty:", cal_uncertainty)
